# hDirect-MAP (v.1.0.1)

**hDirect-MAP, an algorithm that projects T cells into a shared high-dimensional (HD) expression space of diverse T cell functional signatures in which cells group by T cell phenotypes, poised to ICB outcomes, rather than cancer-specific conditions.**

This ipython notebook is part of the submission of _"Robust and Generalizable Predictive Biomarker of Response to Checkpoint Immunotherapy using High-Dimensional Cell Projection with hDirect-MAP."_

Here, we use the dataset of GSE123813 [https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE123813] as an example. This notebook is intended to illustrate a new algorithm for high-dimensional cell projection and its application in prediction of response to ICB. These patient samples were derived from baseline (pre-ICB). These patients received anti-PD-1, or anti-CTLA-4, or anti-PD-1/anti-CTLA-4.



Python libraries that are used in this tutorial:
- numpy for scientific computing needed in high-dimensional cell projection
- numba for GPU scientific computing (optional, only needed for GPU users)
- Matplotlib and seaborn for figures and data mining models
- conda and numba installation instruction (cuda.ipynb)

R libraries that are used in this tutorial:
- ggplot2 for figures
- RColorBrewer, viridis, and circlize for visualization in R
- ComplexHeatmap and scales for heatmap plotting


## Table of contents:

  * <a href=#CD8>1 CD8 and CD4 T cell data</a>
    * <a href=#CD8>1.1 Exhaustion,effector, and memory markers</a>
    * <a href=#preprocessing>1.2 data preprocessing for functional markers</a>
  * <a href=#clustering>2 Hierarchical clustering and high-variance markers</a>
  * <a href=#PO>3 High-dimensional cell projection by the PO model</a>
    * <a href=#PO>3.1 the Pareto Optimization model</a>
    * <a href=#criteria>3.2 Setting Optimization Criteria for different T cell functional signatures</a>
    * <a href=#hdirectmap>3.3 hDirect-MAP</a>
      * <a href=#hdirectmap>3.3.1 Update parameters required for hDirect-MAP</a>
      * <a href=#GC>3.3.2 Implementation on Google Cloud Virtual machine (optional)</a>
      * <a href=#local>3.3.3 Implementation on local computer</a>
      * <a href=#fb>3.3.4 Frontier and Back annotation</a>
      * <a href=#combine>3.3.5 combine input and output</a>
    * <a href=#hdmetrics>3.4 HD metrics and HD phenotypes</a>
      * <a href=#heatmap>3.4.1 Visualize HD metrics and expression data by heatmap</a>
      * <a href=#monitoring>3.4.2 Monitoring HD-low and HD-high cells</a>
    * <a href=#prediction>3.5 Prediction of response to ICB</a>
      * <a href=#dataset>3.5.1 Datasets</a>
      * <a href=#random>3.5.2 Random Forest Regressor Model training and testing</a>
      * <a href=#poised>3.5.3 Poised metrics for rediction of response to ICB</a>

<a id='CD8' ></a>

# 1 CD8 and CD4 T cell data

## 1.1 Exhaustion,effector, and memory markers
We established a shared signature database for candidate T cell functions, based on the Gene Set Enrichment Analysis (GSEA) and Molecular Signatures Database (MSigDB), published literature, and T cell functional markers in CyTOF marker panels. We included 93 exhaustion genes or proteins, 200 effector genes or proteins, and 131 memory genes or proteins (Supplementary Tables 2-4).

The data are in Supplementary Tables or in the "data/signatures" folder

In [29]:
!ls data/signatures/*.csv

data/signatures/eff.csv        data/signatures/mem.csv
data/signatures/exhaustion.csv


In [30]:
!awk '(FNR > 2) {print $0}' data/signatures/exhaustion.csv|wc

      93     575   19075


In [31]:
!awk '(FNR > 2) {print $0}' data/signatures/eff.csv|wc

     200    1319   41188


In [32]:
!awk '(FNR > 2) {print $0}' data/signatures/mem.csv|wc

     131     831   36005


<a id='preprocessing' ></a>

## 1.2 data preprocessing for functional markers
CD8 T cells were identified by the expression level of CD3/CD8, and CD4+ T cells were identified by expression levels of CD3/CD4 and the cell annotation in the datasets.

An example to processing the CD4 and CD8 T cells from the basal cell carcinoma scRNA-seq dataset.

R
```Rscript
mkdir<-function(d){
  if (! file.exists(d)){
    dir.create(d)
  }
}
cancer='bcc'
mkdir(paste0("../data/",cancer,"_scRNA-seq"))
setwd(paste0("../data/",cancer,"_scRNA-seq"))
### data download

#https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE123813

### data loading
# Count data
library(dplyr)
bcc_matrix=gzfile('GSE123813_bcc_scRNA_counts.txt.gz','rt')  
dat=read.csv(bcc_matrix,header=T,sep='\t')

# meta data
GSE123813_bcc_all_metadata <- read.delim("GSE123813_bcc_all_metadata.txt")
GSE123813_bcc_scRNA_cells <- colnames(dat)
sample_bcc=GSE123813_bcc_all_metadata[GSE123813_bcc_all_metadata[,'cell.id'] %in% GSE123813_bcc_scRNA_cells,]

### identify T cells by annotation
CD4<-c("CD4_T_cells")
sample_cd4t=sample_bcc[sample_bcc[,'cluster'] %in% CD4,]

CD8<-c("CD8_mem_T_cells","CD8_ex_T_cells","CD8_act_T_cells")
sample_cd8=sample_bcc[sample_bcc[,'cluster'] %in% CD8,]

### identify CD4 T cells by CD3/CD4 expression
dat_cd4=dat[,dat['CD3D',]>0 | dat['CD3E',]>0]
dat_cd4=dat_cd4[,dat_cd4['CD4',]>0]                   

### identify CD8 T cells by CD3/CD8 expression levels
dat_cd8=dat[,dat['CD8A',]>0 | dat['CD8B',]>0]
dat_cd8=dat_cd8[,dat_cd8['CD3D',]>0|dat_cd8['CD3E',]>0]                

### data normalization at the cell level
#   by total count
#   this step is to remove the batch effects of cells
#   We used the same strategy as Seurat (https://satijalab.org/seurat/) 
#   and Scanpy (https://scanpy.readthedocs.io/en/stable/)


data.countExpressed <-
  apply(dat_cd8, 2, function(x){sum(as.numeric(x[x>0]))})

sample_cd8 <- t(t(dat_cd8) / data.countExpressed)*10000

### Data for the functional signature and variance for the markers
markers<-c('exhaustion','eff','mem')
for (marker in markers){
  signature <- read.csv(paste0("../signatures/",marker,".csv"), header=TRUE, skip=1)
  
  genes<-signature[,'Gene.Symbol']
  dat_cd8_marker<-dat_cd8[genes,]
  
  SD=apply(dat_cd8_marker,1, sd, na.rm = TRUE)
  marker_sd<-data.frame(genes,SD)
  out<-"../Marker_dat/"
  mkdir(out)
  out<-paste0(out,'/',cancer,'_',marker)
  mkdir(out)
  write.table(marker_sd,paste0(out,'/','allmarkers_var.txt'),row.names = FALSE,quote=FALSE,sep='\t')
  write.table(t(dat_cd8_marker),paste0(out,'/','heatmap_data.txt'),row.names = TRUE,quote=FALSE,sep='\t')
}

```

<a id='clustering' ></a>

# 2 Hierarchical clustering and high-variance markers

This step is to identify the highly variable markers with consistent T cell phenotypes. The markers used for high-dimensional cell projection were clustered in the hirarchicall clustering by the complexheatmap package of R

Bash
```bash
home=`pwd`
codes=$home/codes
data=$home/data
cancer='bcc'
current_dir=$data
exp_dir_prefix=${data}'/Marker_dat/'${cancer}'_'
outupt_dir=${data}'/combined_for_heatmap/cluster_'${cancer}
Rscript=/Library/Frameworks/R.framework/Resources/bin/Rscript
$Rscript --vanilla $codes/heatmap_marker_cluster.R \
    $cancer \
    $current_dir \
    $exp_dir_prefix
    
```
exhaustion:
<img src="data/heatmap_cluster/exhaustion.png">
eff:
<img src="data/heatmap_cluster/eff.png">
mem:
<img src="data/heatmap_cluster/mem.png">

<a id='PO' ></a>

# 3 High-dimensional cell projection by the PO model

## 3.1 The Pareto Optimization model

To enable high-dimensional cell projection, hDirect-MAP uses a versatile high-dimensional modeling algorithm, namely Pareto Optimization (PO) model to project T cells into a shared HD expression space comprising diverse T cell functional signatures. 

The PO model is a multiple-objective optimization (MOO) model. The PO model can combine and distinguish the molecules in the diverse T cell functional signatures, without needing to reduce molecular dimensions. Mathematically,

In [43]:
%%latex
\begin{equation*}
\label{MOO}
 \textbf{MOO:} \begin{cases} 
 \underset{\mathbf{x}}{\text{max  }} &\mathbf{y}=\mathbf{f(x)}=\left[f_1(\mathbf{x}), f_2(\mathbf{x}), \ldots, f_k(\mathbf{x})\right]
 \\ \text{subject to }  &\mathbf{x} \in \mathbf{\Omega}
 \end{cases}
 \end{equation*}

 where $\mathbf{\Omega}=\left\{\mathbf{x}=(\mathbf{x}_1,\mathbf{x}_2,\ldots,\mathbf{x}_n) \in \mathbf{X} |
 \mathbf{e(x)} = \left[c_1(\mathbf{x}), c_2(\mathbf{x}), \ldots, c_m(\mathbf{x})\right] \leq \mathbf{0}\right\}$ is called as a feasible set, containing the set of decision vectors $\mathbf{x}$ that satisfies the constraints $\mathbf{e(x)} \leq \mathbf{0}$. 

<IPython.core.display.Latex object>

In [39]:
%%latex
\begin{equation}
\label{PO}
 \textbf{PO:} \begin{cases} \underset{\mathbf{y}}{\text{max}} &\left[\mathbf{y}_1, \mathbf{y}_2, \ldots, \mathbf{y}_k\right]\\ \text{subject to } &\mathbf{y} \in \mathbf{\Theta} \end{cases}
 \end{equation}

<IPython.core.display.Latex object>

In [47]:
%%latex
Within a k-dimensional expression space ($\mathbf{\Theta}$), the PO model uses its $k$ objectives ($\mathbf{y}_i,  i = 1,2,\ldots,k$ ) to integrate $k_1$  molecules from exhaustion signature, $k_2$  molecules from effector signature, and $k_3$  molecules from memory signature. 

<IPython.core.display.Latex object>

<img src="data/workflow/PO.png">

<a id='criteria' ></a>

## 3.2 Setting Optimization Criteria for different T cell functional signatures

Although there exist studies on associations of individual T cell functions and response to ICB (RCB), the complex relationships between diverse T cell functions and RCB are still unknown. hDirect-MAP aims to understand and explain how each T cell functional molecule contributes to RCB and what is the differences between these molecules in determining RCB. We used melanoma baseline (pre-ICB) dataset (patient sample n=18, CD8 T cell number N=2,761, and treatment: anti-PD-1, or anti-CTLA-4, or anti-PD-1/anti-CTLA-4) as our discovery cohort. We found that essential T cell effector molecules such as GZMA, GZMB, and PRF1 are not associated with responders (patients who responded to ICB). Alternatively, effector signature occurs together with exhaustion signature molecules, e.g., PDCD1 (PD-1), CTLA4, and HAVCR2 (TIM-3) in the T cells of non-responders (patients who did not respond to ICB). Interestingly, we discovered that memory signature determines RCB when combined with effector and exhaustion signatures, as indicated by the distinct expression pattern of memory molecules compared to those from effector and exhaustion molecules. In this paper, we considered three fundamental T cell functional signatures, i.e., exhaustion, effector, and memory, in the high dimensional cell projection.

Accordingly, we set maximum for exhaustion and effector signatures and minimum for memory signature.

```bash
exh=1
eff=1
mem=-1
python prepare_3types_spea2_input.py \
        $cancer \
        $exp_dir_prefix \
        $exh \
        $eff \
        $mem
```

<a id='hdirectmap' ></a>

## 3.3 hDirect-MAP

hDirect-MAP is a python software package for implementation of the PO model for high-dimensional cell projection. To enable modeling a large number of cells, the software includes a module for GPU computing using Numba library. 

### 3.3.1 Update parameters required for hDirect-MAP
```bash
cancer=$cancer
samp="3types_spea2_"${cancer}
out="../data/hDirect-MAP_output"
data_folder=../data_folder
ratio=0.8
GPU='TRUE' ### or set GPU="" if using CPUs
GC='TRUE' ### specify if google cloud virtual machine is used
mkdir $data_folder
python change_config.py $samp $out $ratio $data_folder $GPU
hdirectmap_codes=hdscmed_CD4
cp -rf ../$samp ./data_folder/
cp -f config.py $hdirectmap_codes/
```

<a id='GC' ></a>

### 3.3.2 Implementation on Google Cloud virtual machine (optional)

If using a virtual machine with GPUs on Google Cloud, hDirect-MAP can automatically implement the codes and data on Google Cloud. 

```bash

if (( $GC == 'TRUE' )); then

#### cp codes with data to gc
virture_machine="guangxujin@gpus-conda"
zone="us-east1-b"
bash move_data_to_gc.sh ../run_gc $virture_machine
bash move_data_to_gc.sh run_hDiretmap.sh $virture_machine

#### run codes on gc
samp_folder=${data_folder}/${samp}
cmd_bash=run_hDiretmap.sh
codes_folder=run_gc/${hdirectmap_codes}
cmd='bash '${cmd_bash}' '${samp_folder}' '${codes_folder}
echo $cmd
gcloud compute ssh $virture_machine --zone=$zone --command=$cmd

#### return results to local
folder_gc=run_gc/data_folder/result__${samp}__${ratio}
des_local=data_folder
bash get_data_from_gc.sh $folder_gc $des_local $virture_machine 

fi

```

<a id='local' ></a>

### 3.3.3 Implementation on local computer

```bash
if (( $GC != 'TRUE' )); then
samp_folder=${data_folder}/${samp}
cmd_bash=run_hDiretmap.sh
codes_folder=run_gc/${hdirectmap_codes}

bash ${cmd_bash} \
        ${samp_folder} \
        ${codes_folder}

fi

```

<a id='fb' ></a>

### 3.3.4 Frontier and Back annotation

In [55]:
%%latex

\begin{equation}
\text{A T cell } \mathbf{a}  \text{ belongs to } \mathbf{Frontier} \Leftrightarrow S(\mathbf{a}) > 0 \text{ and } F(\mathbf{a}) = 0 
\end{equation}

\begin{equation}
\text{A T cell } \mathbf{a}  \text{ belongs to } \mathbf{Back} \Leftrightarrow S(\mathbf{a}) = 0 \text{ and } F(\mathbf{a}) > 0 
\end{equation}

<IPython.core.display.Latex object>

Bash
```bash
#### identify back and frontier
data_folder='data_folder'
output=$data_folder/result__${samp}__${ratio}
python identify_back_front.py $output
```

<a id='combine' ></a>

### 3.3.5 combine input and output

Bash
```bash

#### combine input and output
data_folder='data_folder'
samp_folder=${data_folder}/${samp}
input_folder=$samp_folder
output=$data_folder/result__${samp}__${ratio}
output_folder=${output}/$cancer
immune_marker_folder='../Marker_dat'
combined_data_folder='../combined_input_spea2_data'
mkdir $combined_data_folder
combined_folder=${combined_data_folder}/"exp_spea2_"${cancer}
python combine_input_output.py \
            $input_folder \
            $output_folder \
            $immune_marker_folder \
            $cancer \
            $combined_folder


```

<a id='hdmetrics' ></a>

## 3.4 HD metrics and HD phenotypes

<a id='heatmap' ></a>

### 3.4.1 Visualize HD metrics and expression data by heatmap

```bash

current_dir=`pwd`
combined_data_dir=$combined_folder
combined_data_outdir=${combined_data_folder}/"heatmap_prediction_"${cancer}
Rscript=/Library/Frameworks/R.framework/Resources/bin/Rscript
$Rscript --vanilla visualize_data.R \
        $current_dir \
        $combined_data_dir \
        $combined_data_outdir \
        $cancer
    
```

<img src="data/workflow/heatmap.png">

<a id='monitoring' ></a>

### 3.4.2 Monitor HD-low and HD-high cells

<a id='prediction' ></a>

## 3.5 Prediction of response to ICB

Using 5-fold cross-validation, we implemented RCB prediction for each of the scRNA-seq and CyTOF datasets. The classification power was evaluated by the area under the receiver operating characteristic curve (AUC). To intuitively display the association of the four HD metrics and RCB, we used the predictive probability as a biomarker, which we called poised metric, indicating the extent to which T cells of a patient sample are poised to HD-high and the probability of a patient who is classified as a non-responder (NR).

<a id='dataset' ></a>

### 3.5.1 Datasets

We applied hDirect-MAP to the CD8+ T cell data from four publically available scRNA-seq datasets for basal cell carcinoma (BCC, sample number, n=28, cell number, N=13,832), squamous cell carcinoma (SCC, n=11, N=12,029), and melanoma (Baseline or pre-ICB: n=18, N=2,761; and post-ICB: n=35, N=4,628) and two publically available CyTOF datasets from melanoma tissues (n=11, N=101,257) and melanoma peripheral blood mononuclear cells (PBMC, n=24, N=354,180). These patient samples were derived from baseline (pre-ICB) or post-ICB. These patients received anti-PD-1, or anti-CTLA-4, or anti-PD-1/anti-CTLA-4.


<img src="data/workflow/sample.png">

<a id='random' ></a>

### 3.5.2 Random Forest Regressor Model training and testing
```bash

test_per=0.2
combined_data=${combined_data_folder}/"heatmap_prediction_"${cancer}
output_dir=${combined_data_folder}/"HDmetrics_predictionmodel_roc_"${cancer}
sample_name='split'
python roc_prediction_training.py \
            $cancer \
            $test_per \
            $combined_data \
            $output_dir \
            $sample_name
```

<a id='poised' ></a>

### 3.5.3 Poised metrics for rediction of response to ICB

```bash

python roc_prediction_testing.py \
        $cancer \
        $test_per \
        $combined_data \
        $output_dir \
        $sample_name
```

<img src="data/workflow/prediction.png">